# Multi-Channel Timeseries (Small, In-Memory)

Insert text about this focusing on using a numpy array approach without any downsampling

In [ ]:
import numpy as np; np.random.seed(0)
import pandas as pd
from scipy.stats import zscore
import string

import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import opts
from holoviews import Dataset
from bokeh.models import HoverTool
import panel as pn; pn.extension(template='fast')
from holonote.annotate import Annotator
from holonote.app import PanelWidgets

## Generate fake data

In [ ]:
n_channels = 8
n_seconds = 300
fs = 256  # Sampling frequency

init_freq = .01  # Initial sine wave frequency in Hz
freq_inc = 2/n_channels  # Frequency increment
amplitude = 1

total_samples = n_seconds * fs
time = np.linspace(0, n_seconds, total_samples)
channels = [f'CH {i}' for i in range(n_channels)]
groups = ['EEG'] * (n_channels // 2) + ['MEG'] * (n_channels - n_channels // 2)

data = np.array([amplitude * np.sin(2 * np.pi * (init_freq + i * freq_inc) * time)
                 for i in range(n_channels)])
print(f'shape: {data.shape} (n_channels, samples) ')

## Visualize multi-channel timeseries

In [ ]:
time_dim = hv.Dimension('Time', unit='s')
amplitude_dim = hv.Dimension('Amplitude', unit='µV')

# set group colors
color_map = dict(zip(set(groups), cc.b_glasbey_bw_minc_20[::-1][:len(set(groups))]))
group_color_opts = [opts.Curve(grp, color=grpclr) for grp, grpclr in color_map.items()]

# Create curves overlay plot
curves = []
for group, channel, channel_data in zip(groups, channels, data):
    ds = Dataset((time, channel_data), [time_dim, amplitude_dim])
    curve = hv.Curve(ds, time_dim, amplitude_dim, group=group, label=f'{channel}')
    curve.opts(
        subcoordinate_y=True,
        subcoordinate_scale=.75,
        color="black",
        line_width=1,
        tools=['hover'],
        hover_tooltips=[("Group", "$group"), ("Channel", "$label"), "Time", "Amplitude"],
        )
    curves.append(curve)

curves_overlay = hv.Overlay(curves, kdims="Channel")

curves_overlay = curves_overlay.opts(
    *group_color_opts,
    opts.Overlay(
    xlabel="Time (s)", ylabel="Channel", show_legend=False,
    padding=0, aspect=1.5, responsive=True, shared_axes=False, framewise=False, min_height=100,)
)

# Create minimap
y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]
z_data = zscore(data, axis=1)
minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))
minimap = minimap.opts(
    cmap="RdBu_r",
    colorbar=False,
    xlabel='',
    alpha=0.5,
    yticks=[yticks[0], yticks[-1]],
    toolbar='disable',
    height=120,
    responsive=True,
    default_tools=[],
    )

# Link minimap widget to curves overlay plot
RangeToolLink(minimap, curves_overlay, axes=["x", "y"],
              boundsy=(-.5, 5.5),
              boundsx=(0, time[len(time)//3])
             )

app = pn.Column((curves_overlay + minimap).cols(1), min_height=500).servable()
app


## Add Time-Range Annotations

## Add time-range annotation (Under Construction)

### Create fake time range annotations

In [ ]:
def create_range_annotations(n_total_seconds: int, n_categories: int, 
                             n_total_annotations: int, duration: int = 1) -> pd.DataFrame:

    
    start_times = np.sort(np.random.randint(0, n_total_seconds - duration, n_total_annotations))
    
    # Ensure the annotations are non-overlapping
    for i in range(1, len(start_times)):
        if start_times[i] < start_times[i-1] + duration:
            start_times[i] = start_times[i-1] + duration
    end_times = start_times + duration
    categories = np.random.choice(list(string.ascii_uppercase)[:n_categories], n_total_annotations)
    
    df = pd.DataFrame({
        'start': start_times,
        'end': end_times,
        'category': categories
    })
    df['category'] = df['category'].astype('category')
    return df

np.random.seed(1)
n_categories = 2
n_total_annotations = 5
annotations_df = create_range_annotations(n_seconds, n_categories, n_total_annotations)
annotations_df.sample(5)

In [ ]:

annotator = Annotator({"Time": float}, fields=["category"])
annotator.define_annotations(annotations_df, Time=("start", "end"))

annotations_4_overlay = annotator.get_element("Time")

# Setup Annotator styling and groupby
unique_categories = ["A", "B", "C"]
color_map = dict(zip(unique_categories, cc.glasbey[:len(unique_categories)]))

annotator.style.color = hv.dim("category").categorize(categories=color_map, default="grey")
annotator.groupby = "category"
widget = pn.widgets.MultiSelect(name="Show category", value=["B", "C"], options=["A", "B", "C"], )
annotator.visible = widget
widget.servable(location='sidebar')

annotator_tools = PanelWidgets(annotator, {"category": unique_categories})

# TODO: BUG: adding the annotator tools to the servable app prevents anything from displaying when served
annotator_tools_pn = pn.panel(annotator_tools).servable(target='sidebar')

app_w_annotator = pn.Column((curves_overlay * annotations_overlay + minimap * annotations_overlay).cols(1), min_height=500).servable()